In [1]:
import datetime
import gc
import os
import warnings
warnings.filterwarnings("ignore")
import random
from collections import defaultdict
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import Pool, cv
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score,make_scorer,f1_score
from sklearn.model_selection import KFold, train_test_split

from lightgbm import LGBMClassifier,Dataset
from sklearn.model_selection import GridSearchCV

from dataclasses import dataclass


%matplotlib inline
path_dir=os.getcwd()
path_train='../train_dataset_train.csv'
path_test='../test_dataset_test.csv'
path_subm='../Иннополис/sample_solution.csv'

@dataclass
class Config:
    RS: int = 42
    treshold: float = 0.0
    device='cuda:0'
    num_epochs: int = 120
    batch_size: int = 256
    num_workers: int = 2
        
def get_df(path,train=True,drp=2):
    df=pd.read_csv(path)
    df=df.sort_index(axis=1)
    df.drop(['id','.geo','area'],axis=1,inplace=True)
    if train:
       # df=df.loc[df.crop.isin([0,4])]
        df.reset_index(drop=True,inplace=True) 
        ##    
        drop04=[ 469,  432,  223,  582,  947,  900,   19,  905,  864,  598,  541,
            1097,  269,   43,  455,  413,  584,  473,  281,  804]   
        drop02=[922, 2]
        drop03=[117, 250, 3, 88, 86, 158, 239, 145, 107]
        drop05=[255]
        drop06=[1805, 484]
        drp=list(set(drop02+drop03+drop04+drop05+drop06))
        print(f'droped: {len(drp)}')
        df.drop(drp,axis=0,inplace=True)          
        y=df[['crop']]
        df.drop(['crop'],axis=1,inplace=True)  
    
   # df.rename({c:pd.to_datetime(c.replace('nd_mean_','')) for c in df.columns},axis=1,inplace=True) 
    df[df<=0]=0    
    return (df.reset_index(drop=True),y.crop.reset_index(drop=True).values) if train else df.reset_index(drop=True)

X_train,y_train=get_df(path_train)
X_test=get_df(path_test,False)
df_subm=pd.read_csv(path_subm,index_col='id')

droped: 22


In [14]:
params={      
    'boosting_type':'gbdt',#'gbdt',#'dart',
    'class_weight':'balanced',   
    'objective': 'multiclass',  
    'learning_rate':0.1,
    #'max_depth':5,#
    'min_data_in_leaf':40,#79,57
    'bagging_fraction':50,
}
model_lightgbm=LGBMClassifier(**params)

In [15]:
model_lightgbm.fit(X_train,y_train)

LGBMClassifier(class_weight='balanced', min_data_in_leaf=40,
               objective='multiclass')

In [34]:
params_gr  = { 
    #'min_data_in_leaf':range(20,90),
    #'lambda_l2':[0.5,1,1.5,2,2.5,3,3.5,4],
    'min_child_samples':range(10,70),
    #'min_data_in_leaf',
    #'num_leaves':range(14,31,3),
    #'min_data_in_leaf':range(20,51,5),
    #'reg_lambda':[1,1.5,2,2.5,3]
}

In [35]:
kf=KFold(n_splits=5,shuffle=True,random_state=42)
scorer = make_scorer(f1_score, average = 'weighted')
gs=GridSearchCV(estimator=model_lightgbm,param_grid=params_gr,cv=5,verbose=True,n_jobs=-1,scoring=scorer)
gs.fit(X_train,y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5,
             estimator=LGBMClassifier(class_weight='balanced',
                                      min_data_in_leaf=40,
                                      objective='multiclass'),
             n_jobs=-1, param_grid={'min_child_samples': range(10, 70)},
             scoring=make_scorer(f1_score, average=weighted), verbose=True)

In [36]:
gs.best_params_#57 66 40

{'min_child_samples': 10}

In [37]:
cv_results = pd.DataFrame.from_dict(gs.cv_results_)#0.968759
cv_results.sort_values(by=['mean_test_score'],ascending=False).head(40)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_child_samples,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,8.677994,0.449871,0.054103,0.005926,10,{'min_child_samples': 10},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
1,8.108742,0.148270,0.054819,0.002210,11,{'min_child_samples': 11},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
32,8.737721,0.074786,0.071453,0.005466,42,{'min_child_samples': 42},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
33,8.947959,0.366670,0.062856,0.002752,43,{'min_child_samples': 43},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
34,8.619319,0.103779,0.061375,0.003904,44,{'min_child_samples': 44},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
35,8.291952,0.127512,0.074364,0.007746,45,{'min_child_samples': 45},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
36,8.630817,0.264709,0.062623,0.001297,46,{'min_child_samples': 46},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
37,8.761966,0.385336,0.068770,0.008301,47,{'min_child_samples': 47},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
38,8.624742,0.219674,0.077781,0.031687,48,{'min_child_samples': 48},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1
39,8.579109,0.331975,0.069394,0.009178,49,{'min_child_samples': 49},0.964811,0.968759,0.958433,0.968751,0.974035,0.966958,0.005173,1


In [ ]:
#0.964776